In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0

# Compute activity summary (19.88 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|19,872,262|12|19,872,262|GetValue
|PnLExplainReport:data|1|19,872,249|396|19,872,249|GetValue/Calc
|TradingContainer:NPV|11|19,808,238|138|1,800,748|GetValue
|TradingContainer:NPV|11|19,808,100|1,694|1,800,736|GetValue/Calc
|Portfolio:items|121|19,634,541|1,217|162,268|GetValue
|Portfolio:items|121|19,634,444|29,326|162,268|GetValue/Calc
|Workbook:items|1,104|18,589,410|10,922|16,838|GetValue
|Workbook:items|1,100|18,578,487|9,199,980|16,889|GetValue/Calc
|PnLExplainReport:cutoffs|1|11,150,460|14|11,150,460|GetValue
|PnLExplainReport:cutoffs|1|11,150,446|133|11,150,446|GetValue/Calc
|Root:Clocks|2|11,150,237|58,928|5,575,118|Context
|TradeOpenEvent|1,014|4,515,610|4,515,610|4,453|Db.Get
|TradeOpenEvent:ticket|22,286|4,132,913|112,223|185|GetValue
|TradingTicket|1,014|4,020,689|4,020,689|3,965|Db.Get
|Root:End|1|1,151,924|47|1,151,924|Context
|Root:Activity Portfolio|1|1,084,820|53|1,084,820|Context
|Root:Amend Portfolio|1|1,036,906|44|1,036,906|Context
|Root:Amend MarketData|1|1,011,096|42|1,011,096|Context
|Portfolio:children|242|1,008,567|1,616|4,167|GetValue
|Portfolio:children|121|1,006,950|508,192|8,321|GetValue/Calc
|Root:Start|1|987,930|24|987,930|Context
|Root:Activity MarketData|1|873,403|45|873,403|Context
|Root:Activity Trading|1|869,467|61|869,467|Context
|Root:Start breaks|1|852,000|41|852,000|Context
|Root:Amend Trading|1|849,786|44|849,786|Context
|PortfolioUpdateEvent:children|121|418,592|2,649|3,459|GetValue
|TradingBook|102|382,488|382,488|3,749|Db.Get
|Equity:NPV|15|171,596|186|11,439|GetValue
|Equity:NPV|15|171,409|527|11,427|GetValue/Calc
|MarketInterface:spot|15|158,012|147|10,534|GetValue
|MarketInterface:spot|15|157,865|541|10,524|GetValue/Calc
|ExternalRefData:state|15|149,164|138|9,944|GetValue
|ExternalRefData:state|15|149,026|334|9,935|GetValue/Calc
|RefData:state|15|148,691|134|9,912|GetValue
|RefData:state|15|148,556|64,618|9,903|GetValue/Calc
|_WorkItemEvent:book2|22,286|109,448|102,731|4|GetValue
|Clock:cutoffs|2,480|106,920|9,150|43|GetValue
|_WorkItemEvent:book1|22,286|102,868|102,868|4|GetValue
|Clock:cutoffs|20|97,842|597|4,892|GetValue/Calc
|Clock:parent|20|97,135|176|4,856|GetValue
|Clock:parent|20|96,959|80,179|4,847|GetValue/Calc
|TradeOpenEvent:quantity|22,286|94,223|94,223|4|GetValue
|_WorkItemEvent:item|11,143|77,045|69,480|6|GetValue
|TradeOpenEvent:premium|11,143|73,145|68,426|6|GetValue
|TradeOpenEvent:action|11,143|68,828|68,828|6|GetValue
|Event:amends|11,340|58,869|58,869|5|GetValue
|TradeOpenEvent:unitPrice|11,143|57,195|57,195|5|GetValue
|PortfolioUpdateEvent|12|55,130|55,130|4,594|Db.Get
|TradingPortfolio|10|40,172|40,172|4,017|Db.Get
|TradingBook:clock|2,200|39,587|19,953|17|GetValue
|RefDataUpdateEvent|9|39,122|39,122|4,346|Db.Get
|TradingBook:clock|1,100|19,634|16,171|17|GetValue/Calc
|Clock|5|18,049|18,049|3,609|Db.Get
|Equity:refdata|15|12,870|143|858|GetValue
|Equity:refdata|15|12,726|424|848|GetValue/Calc
|MarketInterface|2|12,197|12,197|6,098|Db.Get
|ClockEvent|2|8,495|8,495|4,247|Db.Get
|MarketInterface:source|15|8,159|125|543|GetValue
|MarketInterface:source|15|8,033|540|535|GetValue/Calc
|Portfolio:clock|242|7,765|2,179|32|GetValue
|Equity|2|7,565|7,565|3,782|Db.Get
|ClockEvent:parent|8|7,545|84|943|GetValue
|MarketDataSource|2|7,299|7,299|3,649|Db.Get
|Portfolio:books|231|6,463|1,704|27|GetValue
|Portfolio:clock|121|5,586|1,691|46|GetValue/Calc
|Portfolio:books|121|5,150|4,316|42|GetValue/Calc
|ForwardCashflow|1|4,719|4,719|4,719|Db.Get
|RootClock|1|4,042|4,042|4,042|Db.Get
|MarketDataSource:clock|30|3,714|268|123|GetValue
|MarketDataSource:clock|15|3,445|215|229|GetValue/Calc
|Entity:clock|40|535|334|13|GetValue
|RefDataUpdateEvent:data|58|283|283|4|GetValue
|ForwardCashflow:NPV|11|267|134|24|GetValue
|RootClock:cutoffs|53|243|186|4|GetValue
|Entity:clock|20|200|200|10|GetValue/Calc
|MarketInterface:sourceName|15|193|137|12|GetValue
|ForwardCashflow:NPV|11|133|133|12|GetValue/Calc
|Equity:assetName|15|103|103|6|GetValue
|RootClock:cutoffs|1|56|30|56|GetValue/Calc
|MarketInterface:sourceName|15|56|56|3|GetValue/Calc
|RootClock:cosmicAll|1|15|9|15|GetValue
|CosmicAll:dbState|1|11|7|11|GetValue
|PnLExplainReport:valuable|2|9|9|4|GetValue
|PnLExplainReport:ts2|2|7|7|3|GetValue
|PnLExplainReport:ts1|2|6|6|3|GetValue
|RootClock:cosmicAll|1|5|5|5|GetValue/Calc
|CosmicAll:dbState|1|4|4|4|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

In [5]:
dba = db.copy()
valuable = dba.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with SummaryMonitor():
    with Context({clock.cutoffs: ts7}):
        print valuable.NPV()
        node = getNode(valuable.NPV)
node.printInputGraph()

40.85


# Compute activity summary (0.18 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|35
|GetValue|185
|GetValue/Calc|24

<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4551430040 @4867030544 T=None>, nIn=3
   <Node: /Global/Equity/f8a92eac-54f5-4e68-b9da-3359f7e8be6d.(Equity:NPV)() in Root:4551430040 @4681890704 T=None>, nIn=2
      <Node: /Global/Equity/f8a92eac-54f5-4e68-b9da-3359f7e8be6d.(Equity:refdata)() in Root:4551430040 @4681890832 T=None>, nIn=1
         <Node: /Global/Equity/f8a92eac-54f5-4e68-b9da-3359f7e8be6d.(Equity:assetName)() in Root:4551430040 @4681890960 T=None>, nIn=0
      <Node: /Global/MarketInterface/GOOG.(MarketInterface:spot)() in Root:4551430040 @4699955792 T=True>, nIn=2
         <Node: /Global/MarketDataSource/source1.GOOG.(ExternalRefData:state)() in Root:4551430040 @4681889552 T=None>, nIn=1
            <Node: /Global/MarketDataSource/source1.GOOG.(RefData:state)() in Root:4551430040 @4681890320 T=None>, nIn=10
               <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:4551430040 @4699956688 T=True>, nIn=2
                  <Node: /Global/Cl

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [6]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        super(DM1, self).__init__()
            
    def prn(self, input, txt):
        print txt
        isEvent = isinstance(input.object(), Event)
        print input.object(), input.methodId() 
        print isEvent, len(input.inputs)
        print input.tweakPoint
        print
        
    def addDep(self, input, output):
        if not input.tweakable:
            # if isinstance(input.object(), Event) and not input.inputs:
            if not input.inputs:
                return

        output.inputs.add(input)
        input.outputs.add(output)


    def calculated(self, node):
        if not node.isSimplified:
            return
        for input in node.inputs:
            if input not in node.ctx.tweaks:
                addFootnote(text='context simplification failure', info=str(input.key))
                        
    def getNode(self, ctx, key, tweakable=False):
        n = ctx._get(key)
        if n:
            return n
        if key.fullName() in ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items'):
            obj = key.object()
            ts = obj.clock().cutoffs()
            cKey = (obj.clock(), ts)
            if cKey not in self.contexts:
                self.contexts[cKey] = Context({obj.clock().cutoffs: ts})
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key, tweakable)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key, tweakable)
        return ret
    
setDependencyManager(DM1())

db4 = db.copy()
valuable = db4.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with Context({clock.cutoffs: ts7}):
    print valuable.NPV()
    node = getNode(valuable.NPV)
    node.printInputGraph()

40.85
<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4551430040 @4810058576 T=None>, nIn=3
   <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:4551430040 @4810059664 T=True>, nIn=2
      <Node: /Global/Clock/Trading.(Clock:parent)() in Root:4551430040 @4810059344 T=None>, nIn=1
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4551430040 @4810058384 T=True>, nIn=0
      <Node: /Global/Clock/Workflow.(Clock:cutoffs)() in Root:4551430040 @4847516432 T=True>, nIn=2
         <Node: /Global/Clock/Workflow.(Clock:parent)() in Root:4551430040 @4847516304 T=None>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4551430040 @4810058384 T=True>, nIn=0
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4551430040 @4810058384 T=True>, nIn=0
   <Node: /Global/Equity/f8a92eac-54f5-4e68-b9da-3359f7e8be6d.(Equity:NPV)() in Root:4551430040 @4680462864 T=None>, nIn=1
      <Node: /Global/MarketInterface/GOOG.(Mar

In [7]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|0.00
|activity: MarketData|5240.00
|activity: Portfolio|1016.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0

# Compute activity summary (11.37 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|29394
|GetValue/Calc|824


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|11,361,942|19|11,361,942|GetValue
|PnLExplainReport:data|1|11,361,922|430|11,361,922|GetValue/Calc
|TradingContainer:NPV|11|11,354,182|128|1,032,198|GetValue
|TradingContainer:NPV|11|11,354,054|2,094|1,032,186|GetValue/Calc
|PnLExplainReport:cutoffs|1|11,264,296|18|11,264,296|GetValue
|PnLExplainReport:cutoffs|1|11,264,277|142|11,264,277|GetValue/Calc
|Root:Clocks|2|11,264,055|2,342|5,632,027|Context
|Portfolio:items|41|11,205,550|332|273,306|GetValue
|Portfolio:items|33|11,205,491|20,266|339,560|GetValue/Calc
|Workbook:items|301|10,507,129|2,641|34,907|GetValue
|Workbook:items|200|10,504,487|1,661,529|52,522|GetValue/Calc
|TradeOpenEvent|1,014|4,597,417|4,597,417|4,533|Db.Get
|TradeOpenEvent:ticket|4,052|4,087,635|30,755|1,008|GetValue
|TradingTicket|1,014|4,056,879|4,056,879|4,000|Db.Get
|Portfolio:children|66|636,379|475|9,642|GetValue
|Portfolio:children|33|635,903|142,522|19,269|GetValue/Calc
|PortfolioUpdateEvent:children|33|437,810|1,461|13,266|GetValue
|TradingBook|102|403,865|403,865|3,959|Db.Get
|Equity:NPV|15|121,293|161|8,086|GetValue
|Equity:NPV|15|121,131|567|8,075|GetValue/Calc
|MarketInterface:spot|15|110,964|145|7,397|GetValue
|MarketInterface:spot|15|110,818|740|7,387|GetValue/Calc
|ExternalRefData:state|15|102,043|130|6,802|GetValue
|ExternalRefData:state|15|101,912|1,008|6,794|GetValue/Calc
|Clock:cutoffs|903|86,164|3,475|95|GetValue
|Clock:cutoffs|16|82,748|521|5,171|GetValue/Calc
|Clock:parent|16|82,133|161|5,133|GetValue
|Clock:parent|16|81,972|65,039|5,123|GetValue/Calc
|Root:Amend Portfolio|1|57,126|41|57,126|Context
|RefData:state|15|56,376|81|3,758|GetValue
|RefData:state|4|56,294|17,908|14,073|GetValue/Calc
|PortfolioUpdateEvent|12|54,711|54,711|4,559|Db.Get
|TradingPortfolio|10|39,735|39,735|3,973|Db.Get
|RefDataUpdateEvent|9|38,143|38,143|4,238|Db.Get
|_WorkItemEvent:book2|4,052|29,122|21,870|7|GetValue
|_WorkItemEvent:item|2,026|26,722|17,409|13|GetValue
|_WorkItemEvent:book1|4,052|21,575|21,575|5|GetValue
|Clock|5|20,450|20,450|4,090|Db.Get
|TradeOpenEvent:quantity|4,052|19,465|19,465|4|GetValue
|TradeOpenEvent:premium|2,026|18,781|14,676|9|GetValue
|TradeOpenEvent:action|2,026|14,749|14,749|7|GetValue
|TradingBook:clock|1,004|14,573|6,645|14|GetValue
|Root:End|1|13,526|51|13,526|Context
|Event:amends|2,083|12,366|12,366|5|GetValue
|Root:Start|1|12,175|28|12,175|Context
|TradeOpenEvent:unitPrice|2,026|11,676|11,676|5|GetValue
|Equity:refdata|15|9,599|157|639|GetValue
|Equity:refdata|15|9,442|603|629|GetValue/Calc
|Equity|2|9,313|9,313|4,656|Db.Get
|ClockEvent|2|8,871|8,871|4,435|Db.Get
|MarketInterface|2|8,748|8,748|4,374|Db.Get
|MarketInterface:source|15|8,033|123|535|GetValue
|TradingBook:clock|300|7,928|3,590|26|GetValue/Calc
|MarketInterface:source|15|7,910|557|527|GetValue/Calc
|ClockEvent:parent|6|7,394|72|1,232|GetValue
|MarketDataSource|2|7,163|7,163|3,581|Db.Get
|MarketDataSource:clock|41|6,377|1,162|155|GetValue
|Root:Amend MarketData|1|5,668|42|5,668|Context
|Portfolio:clock|283|5,659|1,469|19|GetValue
|MarketDataSource:clock|15|5,215|150|347|GetValue/Calc
|RootClock|1|4,331|4,331|4,331|Db.Get
|Portfolio:clock|41|4,189|463|102|GetValue/Calc
|ForwardCashflow|1|4,105|4,105|4,105|Db.Get
|Portfolio:books|63|3,024|467|48|GetValue
|Portfolio:books|33|2,662|2,100|80|GetValue/Calc
|Root:Activity Trading|1|1,747|44|1,747|Context
|Root:Activity Portfolio|1|906|43|906|Context
|Root:Start breaks|1|653|41|653|Context
|Root:Amend Trading|1|571|37|571|Context
|Entity:clock|32|508|331|15|GetValue
|Root:Activity MarketData|1|464|38|464|Context
|ForwardCashflow:NPV|11|257|120|23|GetValue
|RootClock:cutoffs|43|218|158|5|GetValue
|MarketInterface:sourceName|15|189|137|12|GetValue
|Entity:clock|16|176|176|11|GetValue/Calc
|ForwardCashflow:NPV|11|137|137|12|GetValue/Calc
|Equity:assetName|15|91|91|6|GetValue
|RefDataUpdateEvent:data|15|75|75|5|GetValue
|RootClock:cutoffs|1|60|31|60|GetValue/Calc
|MarketInterface:sourceName|15|52|52|3|GetValue/Calc
|RootClock:cosmicAll|1|15|10|15|GetValue
|PnLExplainReport:valuable|2|12|12|6|GetValue
|CosmicAll:dbState|1|12|9|12|GetValue
|PnLExplainReport:ts1|2|11|11|5|GetValue
|PnLExplainReport:ts2|2|10|10|5|GetValue
|RootClock:cosmicAll|1|5|5|5|GetValue/Calc
|CosmicAll:dbState|1|3|3|3|GetValue/Calc